# ES 위경도 검색

위경도 검색 또는 보여주기 위한 라이브러리를 설치
- elasticsearch: 엘라스틱서치 검색 라이브러리
- folium: 지도(맵)뷰


In [4]:
!pip install Elasticsearch
!pip install folium

from elasticsearch import Elasticsearch


In [5]:
es = Elasticsearch("http://172.16.101.152:9200/")
print(es)

<Elasticsearch([{'host': '172.16.101.152', 'port': 9200}])>


In [6]:
base_lat=33.430262
base_lon=126.929587
query={
        "script_score": {
            "query": {
                "bool": {
                    "filter": [
                        {
                            "geo_distance": {
                                "distance": "3km",
                                "location": {
                                    "lat": 33.430262,
                                    "lon": 126.929587
                                }
                            }
                        }
                    ]
                }
            },
            "script": {
                "lang": "painless",
                "source": "params.month*doc['segment.default.monthly'].value + params.leadTime*doc['segment.default.lt10'].value + params.profit*doc['segment.default.profit'].value + params.priceship*doc['priceship'].value + params.promotion*doc['promotion'].value",
                "params": {
                    "month": 120,
                    "leadTime": 0.5,
                    "profit": 0.0001,
                    "priceship": 0.5,
                    "promotion": 0.5
                }
            }
        }
    }

res = es.search(index="p-hotel", size=100, query=query)

In [7]:
import folium

m = folium.Map([base_lat, base_lon],zoom_start=12)

folium.Marker([base_lat, base_lon],tooltip = "기준").add_to(m)

for result in res["hits"]["hits"]:
    #print(result)
    sub_lat = result["_source"]["location"]["lat"]
    sub_long =  result["_source"]["location"]["lon"]
    
    title =  result["_source"]["name"]
    
    #지도에 데이터 찍어서 보여주기
    folium.CircleMarker([sub_lat,sub_long],color="red",radius = 2,tooltip = title).add_to(m)

#한글이 안나오는 오류로 html로 trouble shooting 
m.save('example.html')
m

In [8]:
query_withExclude={
        "script_score": {
            "query": {
                "bool": {
                    "must_not": [
                        {
                            "geo_shape": {
                                "ignore_unmapped": True,
                                "location": {
                                    "relation": "INTERSECTS",
                                    "shape": {
                                        "coordinates": [
                                            [
                                                [
                                                    126.91257,
                                                    33.44728
                                                ],
                                                [
                                                    126.91011,
                                                    33.46256
                                                ],
                                                [
                                                    126.88917,
                                                    33.45448
                                                ],
                                                [
                                                    126.88884,
                                                    33.44324
                                                ],
                                                [
                                                    126.91257,
                                                    33.44728
                                                ]
                                            ]
                                        ],
                                        "type": "Polygon"
                                    }
                                }
                            }
                        },
                        {
                            "geo_shape": {
                                "ignore_unmapped": True,
                                "location": {
                                    "relation": "INTERSECTS",
                                    "shape": {
                                        "coordinates": [
                                            [
                                                [
                                                    126.91464,
                                                    33.45054
                                                ],
                                                [
                                                    126.91502,
                                                    33.44726
                                                ],
                                                [
                                                    126.91829,
                                                    33.44691
                                                ],
                                                [
                                                    126.9209,
                                                    33.45019
                                                ],
                                                [
                                                    126.91801,
                                                    33.45169
                                                ],
                                                [
                                                    126.91464,
                                                    33.45054
                                                ]
                                            ]
                                        ],
                                        "type": "Polygon"
                                    }
                                }
                            }
                        }
                    ],
                    "filter": [
                        {
                            "geo_distance": {
                                "distance": "3km",
                                "location": {
                                    "lat": 33.430262,
                                    "lon": 126.929587
                                }
                            }
                        }
                    ]
                }
            },
            "script": {
                "lang": "painless",
                "source": "params.month*doc['segment.default.monthly'].value + params.leadTime*doc['segment.default.lt10'].value + params.profit*doc['segment.default.profit'].value + params.priceship*doc['priceship'].value + params.promotion*doc['promotion'].value",
                "params": {
                    "month": 120,
                    "leadTime": 0.5,
                    "profit": 0.0001,
                    "priceship": 0.5,
                    "promotion": 0.5
                }
            }
        }
    }

res_withExclude = es.search(index="p-hotel", size=100, query=query_withExclude)

m2 = folium.Map([base_lat, base_lon],zoom_start=12)
folium.Marker([base_lat, base_lon],tooltip = "기준").add_to(m2)

for result in res_withExclude["hits"]["hits"]:
    #print(result)
    sub_lat = result["_source"]["location"]["lat"]
    sub_long =  result["_source"]["location"]["lon"]
    
    title =  result["_source"]["name"]
    
    #지도에 데이터 찍어서 보여주기
    folium.CircleMarker([sub_lat,sub_long],color="red",radius = 2,tooltip = title).add_to(m2)

#한글이 안나오는 오류로 html로 trouble shooting 
m2.save('example.html')
m2